In [1]:
from sklearn.svm import LinearSVC
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
import os

In [2]:

model_path = "emotiondetector.h5"
feature_extractor_model = load_model(model_path)
feature_extractor_model.layers.pop()  # Remove the last layer (classifier) since you want features

IMG_SIZE = 224
test_transforms = image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

DATA_DIR = "RAVDESS"
emotion_to_index = {'Neutral':0, 'Happiness':1, 'Sadness':2, 'Suprise':3, 'Fear':4, 'Disgust':5, 'Anger':6}


In [3]:
def get_features(data_dir):
    filename2features = {}
    for filename in os.listdir(data_dir):
        frames_file = os.path.join(data_dir, filename)
        for frames_dir in os.listdir(frames_file):
            X_global_features, X_isface = [], []
            imgs = []
            frames_full_dir = os.path.join(frames_file, frames_dir)
            for img_name in os.listdir(frames_full_dir):
                img_path = os.path.join(frames_full_dir, img_name)
                img = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
                img_array = image.img_to_array(img)
                imgs.append(img_array)
                X_isface.append(filename)
                if len(imgs) == 20:
                    img_stack = np.stack(imgs)
                    scores = feature_extractor_model.predict(img_stack)
                    X_global_features.append(scores)  # Append instead of overwriting

            X_isface = np.array(X_isface)
            filename2features[frames_dir] = (X_global_features, X_isface)
            
            # Print dimensions for verification
            print(f"Filename: {frames_dir}, Features Shape: {np.shape(X_global_features)}")

    return filename2features


In [4]:
filename2features = get_features(DATA_DIR)


1/1 [==============================] - 0s 266ms/step
Filename: 01_1, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 80ms/step
Filename: 01_10, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 78ms/step
Filename: 01_100, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 78ms/step
Filename: 01_101, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 80ms/step
Filename: 01_102, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 87ms/step
Filename: 01_103, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 79ms/step
Filename: 01_104, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 72ms/step
Filename: 01_105, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 79ms/step
Filename: 01_106, Features Shape: (1, 20, 7)
1/1 [==============================] - 0s 83ms/step
Filename: 01_107, Features Shape: (1, 20, 7)
1/1 [===========================

In [5]:
def create_dataset(filename2features, data_dir):
    x = []
    y = []
    for filename in os.listdir(data_dir):
        print(filename)
        frames_file = os.path.join(data_dir, filename)
        for frames_dir in os.listdir(frames_file):
            features = filename2features[frames_dir]
            total_features = None
            cur_features = features[0]
            mean_features = (np.mean(cur_features, axis=0))
            std_features = (np.std(cur_features, axis=0))
            max_features = (np.max(cur_features, axis=0))
            min_features = (np.min(cur_features, axis=0))

            # join several features together
            feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)
            total_features = feature

            if total_features is not None:
                x.append(total_features)
                y.append(filename)
    x = np.array(x)
    y = np.array(y)
    print(x.shape, y.shape)
    return x, y


In [6]:
x, y = create_dataset(filename2features, DATA_DIR)

# Normalizing the features
x_norm = preprocessing.normalize(x, norm='l2')

# Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.25, random_state=42)
# Print unique labels in training and testing data
print("Training labels:", np.unique(y_train))
print("Testing labels:", np.unique(y_test))

0
1
2
3
4
5
6
(2496, 560) (2496,)
Training labels: ['0' '1' '2' '3' '4' '5' '6']
Testing labels: ['0' '1' '2' '3' '4' '5' '6']


In [7]:
# Train a Linear SVM with the best parameters
best_clf = LinearSVC(C=10, dual=True)
best_clf.fit(x_train, y_train)

# Make predictions
y_pred = best_clf.predict(x_test)

# Evaluate the model

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Complete accuracy with best parameters: {accuracy}")


Complete accuracy with best parameters: 0.375


In [9]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'dual': [True, False]}
grid_search = GridSearchCV(LinearSVC(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

print("Best parameters found: ", grid_search.best_params_)


C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

Best parameters found:  {'C': 100, 'dual': True}


C:\Users\mamta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### print(metrics.classification_report(y_test, y_pred))
